# Building a Robot that answers all questions about History of Australia

![Alt text](Robot.jpg)

In [3]:
# load the environment variables
%pip install -qU langchain langchain_community langchain_huggingface langchain_openai
from dotenv import load_dotenv
import os

# loads the .env file (if you have a global environment variable, you can skip this)
load_dotenv()

# lets just validate that we have the environment variable
api_key = os.environ.get("OPENAI_API_KEY")

if not api_key:
    raise ValueError("OPENAI_API_KEY is not set in the environment variables")

Note: you may need to restart the kernel to use updated packages.


In [4]:
from langchain_community.document_loaders import WebBaseLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.vectorstores import InMemoryVectorStore

# Using the Wikipedia's infomation about history of Australia
loader = WebBaseLoader("https://en.wikipedia.org/wiki/History_of_Australia")
documents = loader.load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200, separators=["\n\n", "\n", " ", ""])
chunks = text_splitter.split_documents(documents)
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
vectorstore = InMemoryVectorStore.from_documents(chunks, embeddings)

retriever = vectorstore.as_retriever()

In [5]:
# setup the chat model and prompt template
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate

llm = ChatOpenAI(model_name="gpt-4o-mini")

prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant that can answer questions about the History of Australia. Use the provided context to answer the question. IMPORTANT: If you are unsure of the answer, say 'I don't know' and don't make up an answer."),
    ("user", "Question: {question}\nContext: {context}")
])

chain = prompt | llm

In [10]:
query = "Tell me more about European exploration period"

# RETRIEVAL
docs = retriever.invoke(query)
docs_content = "\n\n".join(doc.page_content for doc in docs)

# AUGMENTED + GENERATION
response = chain.invoke({"question": query, "context": docs})

print(response.content)

European exploration of Australia began in the early 17th century predominantly through Dutch expeditions. The first documented European landing was made by Willem Janszoon, who captained the Dutch East India Company ship Duyfken in 1606. Following this, notable explorers included Luis Vaez de Torres, who navigated to the north of Australia later that year, and Dirk Hartog in 1616, who is known for landing on the western shore.

In subsequent years, various explorers contributed to the mapping and understanding of Australia's coasts, such as Abel Tasman, who discovered Van Diemen's Land (now Tasmania) in 1642, and William Dampier, who made landfall on the north-west coast in 1688 and again in 1699. His writings provided influential insights into the continent and its Indigenous peoples.

British exploration became significant in the 18th century, with Lieutenant James Cook's voyages being particularly important. In 1769, Cook set sail to observe the transit of Venus, but his secret ins

In [ ]:
#Create a mini website like chatbox, so can keep asking question over and over again~~~
while True:
    query = input("Enter a question: ")
    if query == "exit":
        break
    docs = retriever.invoke(query)
    response = chain.invoke({"question": query, "context": docs})
    print(response.content)


Enter a question:  tell me about Indigenous history


Indigenous history in Australia dates back tens of thousands of years. The ancestors of Aboriginal Australians are believed to have migrated to the Australian continent between 50,000 and 65,000 years ago, during the last glacial period. They arrived by land bridges and short sea crossings from Southeast Asia. 

Evidence of their presence includes Aboriginal art, which can be traced back approximately 30,000 years, with significant examples found in locations like Ubirr in Kakadu National Park. This extensive history highlights the deep connection Indigenous Australians have with the land and their rich cultural practices, which have been maintained and evolved over thousands of generations. 

If you have any more specific questions about Indigenous history or aspects of it, feel free to ask!
